<a href="https://colab.research.google.com/github/Lukaszc9825/webscrap-and-bigdata-analyze/blob/collab/Otodom_data_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Libraries install and import**

In [131]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup as bs

**Load data**

In [132]:
df1 = pd.read_json('/content/drive/My Drive/data analyze/otodom_rent_data')
df2 = pd.read_json('/content/drive/My Drive/data analyze/otodom_sale_data')

In [133]:
headers = {"User-Agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36\
(KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

In [217]:
df1['Rent'] = 'no data'
df1['Deposit'] = 'no data'
df1['Number of rooms'] = 'no data'
df1['built in'] = 'no data'
df1['floor'] = 'no data'
df1['number of floors'] = 'no data'

In [218]:
df3 = df1[:10]

## *add kolumns with more accurate data*

*scrap each adverisment*

In [222]:
for index,href in enumerate(df3['href']):
  page = requests.get(href, headers = headers)
  soup = bs(page.content, 'html.parser')
  mess = soup.find_all(class_ ='section-overview')
  for m in mess:
    data = m.find('ul')
    data = data.find_all('li')
    temp = {}
    for d in data:
      temp[d.get_text().split(":")[0]] = d.get_text().split(":")[1].strip()
    
    # add rent price to data frame
    if 'Czynsz - dodatkowo' in temp:
      df3.at[index,'Rent'] = "+" + temp.get('Czynsz - dodatkowo')
    elif 'Czynsz' in temp:
      df3.at[index,'Rent'] = temp.get('Czynsz')
    elif 'Czynsz - dodatkowo' in temp == False and 'Czynsz' in temp == False:
      df3.at[index,'Rent'] = 0
    
    # add deposit to data frame
    if 'Kaucja' in temp:
      df3.at[index,'Deposit'] = temp.get('Kaucja')
    elif 'Kaucja' in temp == False:
      df3.at[index,'Deposit'] = 0
    
    # add number of rooms to data frame
    if 'Liczba pokoi' in temp:
      df3.at[index,'Number of rooms'] = temp.get('Liczba pokoi')
    elif 'Liczba pokoi' in temp == False:
      df3.at[index,'Number of rooms'] = 0

    # add year of building to data frame
    if 'Rok budowy' in temp:
      df3.at[index,'built in'] = temp.get('Rok budowy')
    elif 'Rok budowy' in temp == False:
      df3.at[index,'built in'] = 0

    # add floor to data frame
    if 'Piętro' in temp:
      if temp.get('Piętro') == 'parter':
        df3.at[index,'floor'] = '0'
      else:
        df3.at[index,'floor'] = temp.get('Piętro')
    elif 'Piętro' in temp == False:
      df3.at[index,'floor'] = 'no data'

    # add number of floors to data frame
    if 'Liczba pięter' in temp:
      df3.at[index,'number of floors'] = temp.get('Liczba pięter')
    elif 'Liczba pięter' in temp == False:
      df3.at[index,'number of floors'] = 0        

In [223]:
df3

,href,area,advertisment name,for rent/sale,localization,price,Rent,Deposit,Number of rooms,built in,floor,number of floors
0,https://www.otodom.pl/oferta/nowoczesny-wyjatk...,"87,33 m²","Nowoczesny, wyjątkowy apartament na Muranowie",rent,"Warszawa, Śródmieście",5 500 zł,+300 zł,5 500 zł,3,2010,3,7
1,https://www.otodom.pl/oferta/mieszkanie-jagiel...,57 m²,Mieszkanie Jagiellońska w centrum Zakopanego,rent,"Zakopane, tatrzański, małopolskie",3 000 zł,+625 zł,3 000 zł,3,no data,0,no data
2,https://www.otodom.pl/oferta/na-ok-4-miesiace-...,45 m²,"na ok 4 miesiące wynajmę pracownikom, tez Ukraina",rent,"Szczecin, zachodniopomorskie",1 100 zł,+400 zł,1 500 zł,2,no data,8,10
3,https://www.otodom.pl/oferta/rodzinne-mieszkan...,"79,50 m²",Rodzinne mieszkanie w Wilanowie Sarmacka,rent,"Warszawa, Wilanów",4 000 zł,no data,no data,3,2009,2,3
4,https://www.otodom.pl/oferta/mieszkanie-2-poko...,"66,47 m²","Mieszkanie 2 pokoje + salon/kuchnia, centrum W...",rent,"Poznań, Wilda, Górna Wilda",2 250 zł,+490 zł,2 250 zł,2,no data,2,4
5,https://www.otodom.pl/oferta/elegancko-wykoncz...,"42,38 m²",Elegancko wykończone i wyposażone M2. Mordor.,rent,"Warszawa, Mokotów, Wyględów",2 500 zł,+500 zł,no data,2,2019,8,8
6,https://www.otodom.pl/oferta/apartament-ordona...,31 m²,Apartament Ordona sypialnia + pokój z aneksem,rent,"Warszawa, Wola",2 150 zł,+350 zł,2 500 zł,2,2020,1,6
7,https://www.otodom.pl/oferta/przestronna-kawal...,35 m²,Przestronna kawalerka po remoncie Lea/Kijowska,rent,"Kraków, Krowodrza",1 350 zł,+350 zł,1 350 zł,1,1960,0,4
8,https://www.otodom.pl/oferta/dwupokojowe-miesz...,50 m²,Dwupokojowe mieszkanie na Ursynowie,rent,"Warszawa, Ursynów",2 500 zł,no data,2 500 zł,2,2010,0,3
9,https://www.otodom.pl/oferta/kawalerka-srodmie...,22 m²,Kawalerka Śródmieście świetna lokalizacja,rent,"Warszawa, Wola, Muranów",1 600 zł,+200 zł,1 600 zł,1,1950,3,4


In [ ]:
df3['href'][1]